In [12]:
import cvxpy as cp
import torch
from cvxpylayers.torch import CvxpyLayer

In [21]:


n, m = 2, 3
x = cp.Variable(n)
A = cp.Parameter((m, n))
b = cp.Parameter(m)
constraints = [x >= 0]
objective = cp.Minimize(0.5 * cp.pnorm(A @ x - b, p=1))
problem = cp.Problem(objective, constraints)
assert problem.is_dpp()

cvxpylayer = CvxpyLayer(problem, parameters=[A, b], variables=[x])
A_tch = torch.randn(m, n, requires_grad=True)
b_tch = torch.randn(m, requires_grad=True)

# solve the problem
solution, = cvxpylayer(A_tch, b_tch)

# compute the gradient of the sum of the solution with respect to A, b
solution.sum().backward()

In [4]:
A_tch.grad

tensor([[ 1.5139e-08,  1.0238e-08],
        [ 1.0998e+01,  7.5741e+00],
        [-2.1939e+01, -1.5109e+01]])

In [5]:
b_tch.grad

tensor([-6.4693e-09, -4.7209e+00,  9.4172e+00])

In [7]:
A_tch

tensor([[ 0.6674, -1.3942],
        [ 0.9303, -1.8197],
        [ 0.5725, -0.8060]], requires_grad=True)

In [8]:
b_tch

tensor([ 1.2441, -0.7523,  0.0406], requires_grad=True)

In [40]:
gradient = []
for i in range(m):
    add = torch.zeros(m)
    add[i]=0.001
    b_more = b_tch + add
    print("b_tch:",b_tch,"\nb_more:", b_more)
    solution_more, = cvxpylayer(A_tch, b_more)
    # e.g.:f(b_tch+[0.001;0;0])=f(b_tch)+grad^T*[0.001;0;0]
    gradient.append((solution_more.sum()-solution.sum()).data/0.001)
print("gradient_from_taylor:",gradient)
print("gradient_from_cvx:",b_tch.grad)

b_tch: tensor([ 1.2441, -0.7523,  0.0406], requires_grad=True) 
b_more: tensor([ 1.2451, -0.7523,  0.0406], grad_fn=<AddBackward0>)
b_tch: tensor([ 1.2441, -0.7523,  0.0406], requires_grad=True) 
b_more: tensor([ 1.2441, -0.7513,  0.0406], grad_fn=<AddBackward0>)
b_tch: tensor([ 1.2441, -0.7523,  0.0406], requires_grad=True) 
b_more: tensor([ 1.2441, -0.7523,  0.0416], grad_fn=<AddBackward0>)
gradient_from_taylor: [tensor(0.), tensor(-4.7207), tensor(9.4173)]
gradient_from_cvx: tensor([-6.4693e-09, -4.7209e+00,  9.4172e+00])


# QP example

In [22]:
import cvxpy as cp
from cvxpylayers.torch import CvxpyLayer

def QP_layer(nz, nineq):
    """Builds the QP layer.

    The optimization problem is of the form
        \hat z =   argmin_z z^T*Q*z + p^T*z
                subject to G*z <= h
    where Q \in S^{nz,nz},
        S^{nz,nz} is the set of all positive semi-definite matrices,
        p \in R^{nz}
        G \in R^{nineq,nz}
        h \in R^{nineq}
    
    Take the matrix square-root of Q：mentioned in paper P19
    (Differentiable Convex Optimization Layers).
    """
    Q_sqrt = cp.Parameter((nz, nz))
    p = cp.Parameter(nz)
    G = cp.Parameter((nineq, nz))
    h = cp.Parameter(nineq)
    z = cp.Variable(nz)
    obj = cp.Minimize(cp.sum_squares(Q_sqrt*z) + p.T@z)
    cons = [ G@z <= h ]
    prob = cp. Problem(obj, cons)
    assert prob.is_dpp()

    layer = CvxpyLayer (prob, 
                        parameters =[Q_sqrt, p, G, h], 
                        variables =[ z ])
    return layer

In [30]:
nx = 2
ncon_ineq = 1
layer = QP_layer(nx,ncon_ineq)

Q_sqrtval = torch . randn ( 2,nx , nx , requires_grad = True )
pval = torch . randn ( 2,nx , requires_grad = True )
Gval = torch . randn ( 2,ncon_ineq , nx , requires_grad = True )
hval = torch . randn ( 2,ncon_ineq , requires_grad = True )
y, = layer ( Q_sqrtval , pval  , Gval , hval )
print(y)

tensor([[ 0.0968,  1.0110],
        [ 0.1041, -0.1462]], grad_fn=<_CvxpyLayerFnFnBackward>)


In [35]:
G.ndim

2

In [36]:
Gval.ndimension()

3

In [50]:
x = torch.ones(2)
print(x.repeat(4,1).size())

x.repeat(4, 2, 1,1).size()

torch.Size([4, 2])


torch.Size([4, 2, 1, 2])